In [1]:
import pandas as pd
import scipy.stats as st
import numpy as np
from math import floor

import subprocess
import re

In [2]:
class PingError(Exception):
    pass

# Returns rtt or raises the PingError.
def ping(hostname):
    try:
        output = subprocess.check_output(["ping", hostname ,"-c", "1"])
    except subprocess.CalledProcessError:
        raise PingError
    
    match = re.search('time=(.+?) ms', str(output))
    if match:
        return float(match.group(1))
    raise PingError

In [9]:
hostA = "google.ru"
hostB = "google.com"

# Probability of type 1 error.
alpha = [0.1, 0.05, 0.01]
# Probability of type 2 error.
beta = 0.05

df = pd.DataFrame(columns=[hostA, hostB])
df = df.append([{hostA: ping(hostA), 
                 hostB: ping(hostB)}])
df

,google.ru,google.com
0,2.354,1.333


In [10]:
while True:
    # Increese the sample size.
    df = df.append([{hostA: ping(hostA), 
                 hostB: ping(hostB)}])
    n = df.count()[0]
    # Average of A and B.
    muA, muB = df.sum() / n
    sA, sB = df.std()
    # Square standard deviation of A and B.
    s2A, s2B = sA*sA, sB*sB
    # Student's distribution degree of freedom.
    k = floor((s2A/n + s2B/n)**2/((s2A/n)**2/(n-1) + (s2B/n)**2/(n-1)))
    # Statistical criterium.
    z = (muB - muA)/np.sqrt(s2A/n + s2B/n)
    # P - value.
    p = 1 - st.t.cdf(z, k)
   
    # The sample size enough to make the decision.
    n_eff = s2A*(st.norm.ppf(1 - alpha[0]) - st.norm.ppf(beta))**2/(muA - muB)**2
    print(f"n_eff: {n_eff}, current n: {n}")
    
    # Difference between A and B.
    print(f"effect: {muB - muA}, {(muB - muA) * 100 / muA}%")
    
    # Which alpha we can accept H1 With?
    for a in alpha:
        print(a, p, p <= a)
        
    # Now we have only rolling standard deviation, which affects n_eff. 
    # For better results we need to run a (A/A) test to calculate
    # the standart deviation.
    if n >= n_eff:
        break
        

n_eff: 3.2383938312610367, current n: 2
effect: -0.5025, -23.53078904237883%
0.1 0.7975064875329937 False
0.05 0.7975064875329937 False
0.01 0.7975064875329937 False
n_eff: 14.48741317291792, current n: 3
effect: -0.2426666666666668, -12.109115103127086%
0.1 0.773415294671072 False
0.05 0.773415294671072 False
0.01 0.773415294671072 False
n_eff: 10.684621456233685, current n: 4
effect: -0.2602500000000003, -13.437459661804583%
0.1 0.848198649966541 False
0.05 0.848198649966541 False
0.01 0.848198649966541 False
n_eff: 11.800426740784362, current n: 5
effect: -0.2290000000000001, -12.08698405995989%
0.1 0.8766316998290804 False
0.05 0.8766316998290804 False
0.01 0.8766316998290804 False
n_eff: 29.37881461211708, current n: 6
effect: -0.13366666666666682, -7.146039383409079%
0.1 0.7793355011643244 False
0.05 0.7793355011643244 False
0.01 0.7793355011643244 False
n_eff: 23.198397841751433, current n: 7
effect: -0.1375714285714289, -7.375354216129296%
0.1 0.8254773427086621 False
0.05 0.82

n_eff: 283.1780908660425, current n: 57
effect: -0.04071929824561349, -2.3739145554407437%
0.1 0.8418120610213093 False
0.05 0.8418120610213093 False
0.01 0.8418120610213093 False
n_eff: 326.8654806520931, current n: 58
effect: -0.03760344827586137, -2.193988411395421%
0.1 0.8265219058962378 False
0.05 0.8265219058962378 False
0.01 0.8265219058962378 False
n_eff: 287.6300149628997, current n: 59
effect: -0.039762711864406386, -2.3213702615251943%
0.1 0.8432475257145896 False
0.05 0.8432475257145896 False
0.01 0.8432475257145896 False
n_eff: 365.66058681436175, current n: 60
effect: -0.035549999999999526, -2.0820115374478734%
0.1 0.8179711725668043 False
0.05 0.8179711725668043 False
0.01 0.8179711725668043 False
n_eff: 395.6327330933231, current n: 61
effect: -0.03399999999999981, -1.9940006922278086%
0.1 0.8108279992129067 False
0.05 0.8108279992129067 False
0.01 0.8108279992129067 False
n_eff: 327.31071473850636, current n: 62
effect: -0.037096774193548177, -2.1742829592936292%
0.1 0

n_eff: 412.5556192100416, current n: 105
effect: -0.03653333333333375, -2.1369283048298393%
0.1 0.8501048363017477 False
0.05 0.8501048363017477 False
0.01 0.8501048363017477 False
n_eff: 446.18306340453006, current n: 106
effect: -0.036216981132076054, -2.1102450500764407%
0.1 0.8414737595539228 False
0.05 0.8414737595539228 False
0.01 0.8414737595539228 False
n_eff: 465.81710344037424, current n: 107
effect: -0.03546728971962687, -2.069710240566362%
0.1 0.8377425220932946 False
0.05 0.8377425220932946 False
0.01 0.8377425220932946 False
n_eff: 477.56499083844585, current n: 108
effect: -0.034870370370370996, -2.035433624109602%
0.1 0.835884548931552 False
0.05 0.835884548931552 False
0.01 0.835884548931552 False
n_eff: 601.8833258051621, current n: 109
effect: -0.0310091743119274, -1.8120798172909338%
0.1 0.8092417397272944 False
0.05 0.8092417397272944 False
0.01 0.8092417397272944 False
n_eff: 747.6870315873476, current n: 110
effect: -0.027890909090909854, -1.632670437915463%
0.1 

n_eff: 3717.0171699619236, current n: 154
effect: -0.014889610389611674, -0.871621184994354%
0.1 0.6706946472794368 False
0.05 0.6706946472794368 False
0.01 0.6706946472794368 False
n_eff: 4800.269512235996, current n: 155
effect: -0.01309677419354971, -0.7675119380244335%
0.1 0.6519413795967441 False
0.05 0.6519413795967441 False
0.01 0.6519413795967441 False
n_eff: 5939.4811158306375, current n: 156
effect: -0.01173717948718056, -0.6879811829068677%
0.1 0.637625380955471 False
0.05 0.637625380955471 False
0.01 0.637625380955471 False
n_eff: 4846.91959354129, current n: 157
effect: -0.01295541401274014, -0.7591081785136563%
0.1 0.652111018727198 False
0.05 0.652111018727198 False
0.01 0.652111018727198 False
n_eff: 4390.420443175671, current n: 158
effect: -0.013569620253166104, -0.7952168301115098%
0.1 0.659837987463456 False
0.05 0.659837987463456 False
0.01 0.659837987463456 False
n_eff: 4851.9573317795075, current n: 159
effect: -0.012867924528303654, -0.7542106407842464%
0.1 0.65

n_eff: 103921.19978207901, current n: 205
effect: -0.002624390243903818, -0.15482682705736434%
0.1 0.537831245155769 False
0.05 0.537831245155769 False
0.01 0.537831245155769 False
n_eff: 73799.33267784644, current n: 206
effect: -0.0031067961165058833, -0.1833075557083725%
0.1 0.5449615484008492 False
0.05 0.5449615484008492 False
0.01 0.5449615484008492 False
n_eff: 26599.216214695527, current n: 207
effect: -0.005173913043479139, -0.3050308733396129%
0.1 0.57483910653206 False
0.05 0.57483910653206 False
0.01 0.57483910653206 False
n_eff: 28104.29844237708, current n: 208
effect: -0.005024038461539648, -0.29631015875993394%
0.1 0.5730103971338526 False
0.05 0.5730103971338526 False
0.01 0.5730103971338526 False
n_eff: 3026397.6357563767, current n: 209
effect: -0.00048325358851775135, -0.028512789821275628%
0.1 0.5070230523648366 False
0.05 0.5070230523648366 False
0.01 0.5070230523648366 False
n_eff: 16792907.40345288, current n: 210
effect: -0.000204761904762929, -0.01208574696948

...

n_eff: 25249.507532568263, current n: 804
effect: -0.028978855721392582, -1.6398000061934743%
0.1 0.6741757212615439 False
0.05 0.6741757212615439 False
0.01 0.6741757212615439 False
n_eff: 25420.381971862516, current n: 805
effect: -0.02886335403726692, -1.6333333567654051%
0.1 0.6737288319448822 False
0.05 0.6737288319448822 False
0.01 0.6737288319448822 False
n_eff: 25551.54770172673, current n: 806
effect: -0.028771712158808915, -1.6284243256680089%
0.1 0.6734119920859267 False
0.05 0.6734119920859267 False
0.01 0.6734119920859267 False
n_eff: 27641.286596188096, current n: 807
effect: -0.027645600991325736, -1.5647575362257737%
0.1 0.6672186541266839 False
0.05 0.6672186541266839 False
0.01 0.6672186541266839 False
n_eff: 27241.597742258706, current n: 808
effect: -0.027830445544554383, -1.5752968349052148%
0.1 0.6684623190582747 False
0.05 0.6684623190582747 False
0.01 0.6684623190582747 False
n_eff: 26903.15661486031, current n: 809
effect: -0.027987639060568448, -1.584257343510

n_eff: 20636.655909517558, current n: 857
effect: -0.031077012835473594, -1.7567130805306366%
0.1 0.6967988414881005 False
0.05 0.6967988414881005 False
0.01 0.6967988414881005 False
n_eff: 20263.49724198049, current n: 858
effect: -0.03134382284382364, -1.771578201940063%
0.1 0.698552495720235 False
0.05 0.698552495720235 False
0.01 0.698552495720235 False
n_eff: 20244.908439836367, current n: 859
effect: -0.031339930151339734, -1.7713549901730814%
0.1 0.6987413421334141 False
0.05 0.6987413421334141 False
0.01 0.6987413421334141 False
n_eff: 20259.418523455784, current n: 860
effect: -0.031310465116280106, -1.7696771178849868%
0.1 0.6987819051264058 False
0.05 0.6987819051264058 False
0.01 0.6987819051264058 False
n_eff: 20304.390739332837, current n: 861
effect: -0.031257839721255065, -1.7669129739719969%
0.1 0.69868600990561 False
0.05 0.69868600990561 False
0.01 0.69868600990561 False
n_eff: 19206.223468499822, current n: 862
effect: -0.03212412993039515, -1.8150689196568062%
0.1 

n_eff: 20813.760845485307, current n: 910
effect: -0.030061538461538806, -1.6963719864220963%
0.1 0.7014021553154559 False
0.05 0.7014021553154559 False
0.01 0.7014021553154559 False
n_eff: 20568.083100065614, current n: 911
effect: -0.030223929747530853, -1.7055696444035693%
0.1 0.7025929526357624 False
0.05 0.7025929526357624 False
0.01 0.7025929526357624 False
n_eff: 20555.057146457664, current n: 912
effect: -0.03021710526315835, -1.7053555515951047%
0.1 0.702751874534582 False
0.05 0.702751874534582 False
0.01 0.702751874534582 False
n_eff: 20899.40214913641, current n: 913
effect: -0.029950711938664387, -1.6903545015540846%
0.1 0.7013244663149814 False
0.05 0.7013244663149814 False
0.01 0.7013244663149814 False
n_eff: 20902.487074025823, current n: 914
effect: -0.02993216630196982, -1.6894088674513925%
0.1 0.7014111861364707 False
0.05 0.7014111861364707 False
0.01 0.7014111861364707 False
n_eff: 21554.817470652593, current n: 915
effect: -0.029460109289618064, -1.663027739544567

n_eff: 18737.085244429345, current n: 962
effect: -0.03090540540540654, -1.7327039938271103%
0.1 0.7164478710749949 False
0.05 0.7164478710749949 False
0.01 0.7164478710749949 False
n_eff: 18436.47233516772, current n: 963
effect: -0.031140186915888712, -1.7459367438524718%
0.1 0.718117647330238 False
0.05 0.718117647330238 False
0.01 0.718117647330238 False
n_eff: 18629.138784957082, current n: 964
effect: -0.030962655601660893, -1.7360089243236128%
0.1 0.7172062830121837 False
0.05 0.7172062830121837 False
0.01 0.7172062830121837 False
n_eff: 18718.694417097115, current n: 965
effect: -0.030872538860104548, -1.7310483557016667%
0.1 0.716841449165779 False
0.05 0.716841449165779 False
0.01 0.716841449165779 False
n_eff: 18834.54328342678, current n: 966
effect: -0.03076190476190588, -1.7250767739276927%
0.1 0.7163438020987049 False
0.05 0.7163438020987049 False
0.01 0.7163438020987049 False
n_eff: 19612.379781351912, current n: 967
effect: -0.03013029989658844, -1.6898134454308449%
0.

PingError: 

# Sources
[Sample spreadshit](https://docs.google.com/spreadsheets/d/1yoQiaoL2cb-0uZ7quAFF5i1-svjg8Mr8iIfhT4dq3Sg/edit#gid=0)

[Article about AB tests](https://habr.com/ru/post/233911/)

[Video about statistical hypothesis and p-value](https://stepik.org/lesson/26214/step/8?unit=8138)

[Video about Student's criterium](https://stepik.org/lesson/Параметрические-критерии-однородности-26463/step/8?unit=8346)

[Video about the first and second type mistakes](https://stepik.org/lesson/Лемма-Неймана-Пирсона-26214/step/10?unit=8138)

[Video about the sufficient sample size](https://stepik.org/lesson/26216/step/12?unit=8140)